# 📘 Використання Azure OpenAI Function Calling

## Вступ

Цей ноутбук демонструє, як використовувати режим **Function Calling** у Microsoft Azure OpenAI з Python SDK `azure.ai.inference`.

Ми розглянемо:
- Що таке Function Calling та його варіанти використання
- Як описати функцію у форматі OpenAI Function Schema
- Як передати її як інструмент (tool) до моделі
- Як обробити відповідь, якщо модель хоче викликати функцію
- Як відповісти від імені функції, щоб модель могла продовжити відповідь
- Як інтегрувати Function Calling в реальний застосунок

> ⚠️ Перед запуском переконайтесь, що у вас є:
> - 🔑 API ключ Azure OpenAI
> - 🌐 Endpoint
> - 📦 Деплоймент моделі GPT (наприклад, `gpt-4`, `gpt-35-turbo`)

## Навчальні цілі

Після завершення цього уроку ви:
- Зрозумієте призначення та переваги Function Calling
- Навчитеся налаштовувати Function Calling у сервісі Azure OpenAI
- Зможете розробляти ефективні функціональні виклики для своїх застосунків
- Зрозумієте повний цикл обробки функціональних викликів

## Розуміння Function Calling

У цьому уроці ми створимо функціонал для освітнього стартапу, який дозволяє користувачам використовувати чат-бота для пошуку технічних курсів. Ми будемо рекомендувати курси, що відповідають їхньому рівню навичок, поточній ролі та технології, яка їх цікавить.

Для цього ми використовуємо комбінацію:
- **Azure OpenAI** для створення інтерактивного чат-досвіду
- **Microsoft Learn Catalog API** для пошуку курсів на основі запиту користувача
- **Function Calling** для обробки запиту користувача та передачі до функції, яка зробить API-запит

### Навіщо використовувати Function Calling?

Якщо ви вже працювали з великими мовними моделями (LLM), ви, напевно, розумієте їх потужність, а також їх обмеження.

Function Calling - це функціональність сервісу Azure OpenAI, яка допомагає подолати такі обмеження:
1. **Послідовний формат відповіді** - отримання структурованих даних
2. **Можливість використовувати дані з інших джерел** у контексті чату

До появи Function Calling відповіді від LLM були неструктурованими та непослідовними. Розробникам доводилося писати складний код валідації, щоб обробляти різні варіанти відповідей.

Користувачі не могли отримати відповіді на запитання типу "Яка зараз погода у Києві?", оскільки моделі обмежені даними, на яких вони навчались.

### Варіанти використання Function Calling

**1. Виклик зовнішніх інструментів**
Чат-боти чудово відповідають на запитання користувачів. Використовуючи Function Calling, чат-боти можуть використовувати повідомлення від користувачів для виконання певних завдань. Наприклад, студент може попросити чат-бота: "Надішли імейл моєму викладачу, що мені потрібна додаткова допомога з цього предмета". Це може викликати функцію `send_email(to: string, body: string)`.

**2. Створення запитів до API або бази даних**
Користувачі можуть знаходити інформацію, використовуючи природну мову, яка перетворюється на форматований запит. Приклад: викладач запитує "Хто з студентів виконав останнє завдання?", що може викликати функцію `get_completed(student_name: string, assignment: int, current_status: string)`.

**3. Створення структурованих даних**
Користувачі можуть взяти блок тексту або CSV і використовувати LLM для вилучення важливої інформації. Наприклад, студент може перетворити статтю з Вікіпедії про мирні угоди для створення AI-карток. Це можна зробити за допомогою функції `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`.

## 🔐 Крок 1: Підключення до Azure OpenAI

Перший крок – ініціалізувати клієнт `ChatCompletionsClient`, вказавши endpoint та ключ доступу.

In [1]:
import os
import json
import requests
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import ChatCompletionsToolDefinition
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# Виберіть модель загального призначення для тексту
deployment = "gpt-4o"


## 🔧 Крок 2: Визначення функції

Функцію описуємо у форматі JSON Schema (OpenAI Function Calling format) і передаємо як `ChatCompletionsToolDefinition`.

> У нашому випадку, функція `search_courses` — імітує пошук курсів з документації Microsoft Learn за заданими критеріями.

### Елементи функціонального виклику

Ось пояснення ключових частин функціонального виклику:

- **name** - Ім'я функції, яку ми хочемо викликати
- **description** - Опис того, як працює функція. Важливо бути конкретним і чітким
- **parameters** - Список значень та форматів, які модель повинна використовувати у відповіді
- **type** - Тип даних властивостей
- **properties** - Список конкретних значень, які модель буде використовувати для відповіді
- **required** - Обов'язкові властивості для виконання функціонального виклику

In [2]:
# 🔧 Визначення функції, яку модель може викликати
functions = [
    ChatCompletionsToolDefinition(
        type="function",
        function={
            "name": "search_courses",
            "description": "Returns a list of training courses from the Microsoft catalog",
            "parameters": {
                "type": "object",
                "properties": {
                    "role": {
                        "type": "string",
                        "description": """User role (for example: developer, student)"""
                    },
                    "product": {
                        "type": "string",
                        "description": "Covered product (Azure, Power BI, etc.)"
                    },
                    "level": {
                        "type": "string",
                        "description": "User experience level"
                    }
                },
                "required": ["role"]
            }
        }
    )
]


## 📤 Крок 3: Надсилання повідомлення користувача

Ми надсилаємо запит від імені користувача (наприклад, "знайди курси для початківця-розробника по Azure"), і модель вирішує, чи слід викликати функцію.

Щоб інтегрувати функцію у виклик Chat Completion API, ми додаємо `tools=functions` до запиту. Встановлення `tool_choice="auto"` дозволяє LLM самостійно вирішити, яку функцію викликати, виходячи з повідомлення користувача.

In [4]:
# 🧠 Створення повідомлення та виклик моделі з інструментами
messages = [
    {
        "role": "user",
        "content": "Find a course for a beginner developer on Azure"
    }
]

response = client.complete(
    model=deployment,
    messages=messages,
    tools=functions,
    tool_choice="auto"
)

response_message = response.choices[0].message
print("📥 Відповідь моделі:")
print(response_message)


📥 Відповідь моделі:
{'annotations': [], 'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"role":"developer","product":"Azure","level":"beginner"}', 'name': 'search_courses'}, 'id': 'call_Jy7o9TdInZsbwayMGn1rsBbi', 'type': 'function'}]}


## ⚙️ Крок 4: Обробка виклику функції (tool_call)

Якщо модель вирішила викликати функцію, ми зчитуємо її аргументи, викликаємо локальну функцію `search_courses`, і передаємо результат назад в модель як `"role": "tool"`.

Після цього модель формує підсумкову відповідь з урахуванням виклику.

### Інтеграція у застосунок

Для інтеграції у реальний застосунок необхідно:
1. Перевірити, чи модель хоче викликати функцію
2. Отримати аргументи з відповіді моделі
3. Викликати відповідну функцію з отриманими аргументами
4. Додати відповідь від функції до історії повідомлень
5. Зробити новий запит до моделі для отримання підсумкової відповіді користувачу

In [5]:
# ⚙️ Обробка function_call (tool_call)
tool_calls = response_message.tool_calls

if tool_calls and len(tool_calls) > 0:
    first_tool_call = tool_calls[0]
    function_name = first_tool_call.function.name
    function_args = json.loads(first_tool_call.function.arguments)

    def search_courses(role, product, level):
        url = "https://learn.microsoft.com/api/catalog/"
        params = {
            "role": role,
            "product": product,
            "level": level
        }
        response = requests.get(url, params=params)
        modules = response.json().get("modules", [])
        results = []
        for module in modules[:5]:
            title = module.get("title")
            url = module.get("url")
            results.append({"title": title, "url": url})
        return json.dumps(results, ensure_ascii=False)
        

    available_functions = {
        "search_courses": search_courses,
    }

    function_to_call = available_functions[function_name]
    function_response = function_to_call(**function_args)

    print("✅ Результат виклику функції:")
    print(function_response)

    # Додаємо відповідь моделі з tool_calls до історії повідомлень
    messages.append({
        "role": "assistant",
        "content": "Відповідай українською",
        "tool_calls": [
            {
                "id": first_tool_call.id,
                "type": "function",
                "function": {
                    "name": first_tool_call.function.name,
                    "arguments": first_tool_call.function.arguments
                }
            }
        ]
    })

    # Додаємо відповідь функції
    messages.append({
        "role": "tool",  # У найновіших версіях API використовується "tool" замість "function"
        "tool_call_id": first_tool_call.id,  # Це критично!
        "name": function_name,
        "content": function_response
    })

    # Отримуємо фінальну відповідь від моделі
    next_response = client.complete(
        model=deployment,
        messages=messages
    )

    print("📤 Остаточна відповідь моделі:")
    print(next_response.choices[0].message.content)
else:
    print("⚠️ Модель не захотіла викликати функцію.")


✅ Результат виклику функції:
[{"title": "Guide AI workload operations with an AI Center of Excellence", "url": "https://learn.microsoft.com/en-us/training/modules/guide-ai-operations-center-excellence/?WT.mc_id=api_CatalogApi"}, {"title": "Develop products with screen reader support", "url": "https://learn.microsoft.com/en-us/training/modules/develop-products-with-screen-reader-support/?WT.mc_id=api_CatalogApi"}, {"title": "Host a web application with Azure App Service", "url": "https://learn.microsoft.com/en-us/training/modules/host-a-web-app-with-azure-app-service/?WT.mc_id=api_CatalogApi"}, {"title": "Deploy to multiple Azure environments by using JSON ARM template features", "url": "https://learn.microsoft.com/en-us/training/modules/modify-azure-resource-manager-template-reuse/?WT.mc_id=api_CatalogApi"}, {"title": "Secure Azure OpenAI authentication and authorization", "url": "https://learn.microsoft.com/en-us/training/modules/secure-azure-openai-authentication-authorization/?WT.mc

## Висновки та практичні завдання

Тепер ви знаєте, як інтегрувати Function Calling у ваші застосунки з Azure OpenAI. Ця потужна функціональність дозволяє створювати більш інтелектуальні та корисні взаємодії з користувачами.

### Завдання для подальшого вивчення:

1. Додайте більше параметрів до функції, які можуть допомогти учням знаходити більше курсів. Перегляньте доступні API параметри [тут](https://learn.microsoft.com/training/support/catalog-api-developer-reference).
2. Створіть інший функціональний виклик, який отримує додаткову інформацію від користувача, наприклад, його рідну мову.
3. Додайте обробку помилок для випадків, коли функціональний виклик та/або API-виклик не повертає відповідних курсів.
4. Розширте функціонал для рекомендації не лише курсів, але й навчальних шляхів (learning paths).

## Висновки та практичні завдання

Тепер ви знаєте, як інтегрувати Function Calling у ваші застосунки з Azure OpenAI. Ця потужна функціональність дозволяє створювати більш інтелектуальні та корисні взаємодії з користувачами.

### Завдання для подальшого вивчення:

1. Додайте більше параметрів до функції, які можуть допомогти учням знаходити більше курсів. Перегляньте доступні API параметри [тут](https://learn.microsoft.com/training/support/catalog-api-developer-reference).
2. Створіть інший функціональний виклик, який отримує додаткову інформацію від користувача, наприклад, його рідну мову.
3. Додайте обробку помилок для випадків, коли функціональний виклик та/або API-виклик не повертає відповідних курсів.
4. Розширте функціонал для рекомендації не лише курсів, але й навчальних шляхів (learning paths).

### Індивідуальне завдання

> Варіант 9 - containers

**Частина 1**: Модифікуйте базовий ноутбук, щоб реалізувати пошук курсів за іншою тематикою (згідно з вашим варіантом)

**Частина 2**: Розширте функціональність базового рішення одним із наступних способів: 

In [6]:
import os
import json
import requests
import traceback
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import ChatCompletionsToolDefinition
from azure.core.credentials import AzureKeyCredential

# 🔐 Налаштування доступу
API_TOKEN = os.environ.get("GITHUB_TOKEN")
ENDPOINT_URL = "https://models.inference.ai.azure.com"
MODEL_ID = "gpt-4o"

ai_client = ChatCompletionsClient(
    endpoint=ENDPOINT_URL,
    credential=AzureKeyCredential(API_TOKEN),
)

# ==========================================
# 🛠 ОПИС ІНСТРУМЕНТІВ (JSON SCHEMA)
# ==========================================

tools_definition = [
    ChatCompletionsToolDefinition(
        type="function",
        function={
            "name": "fetch_ms_learn_courses",
            "description": "Пошук курсів у каталозі Microsoft Learn за темою контейнерів (Docker, Kubernetes).",
            "parameters": {
                "type": "object",
                "properties": {
                    "role": {"type": "string", "description": "Роль користувача (напр. розробник)"},
                    "subject": {"type": "string", "description": "Тематика (containers, docker)"},
                    "level": {"type": "string", "description": "Рівень: beginner, intermediate, advanced"},
                    "min_rating": {"type": "number", "description": "Мінімальний рейтинг (0-5)"},
                    "limit_minutes": {"type": "integer", "description": "Максимальна тривалість у хвилинах"}
                },
                "required": ["role", "subject"]
            }
        }
    ),
    ChatCompletionsToolDefinition(
        type="function",
        function={
            "name": "get_full_course_details",
            "description": "Отримання розширеної інформації про курс за його унікальним ID (UID).",
            "parameters": {
                "type": "object",
                "properties": {
                    "uid": {"type": "string", "description": "ID курсу або модуля"}
                },
                "required": ["uid"]
            }
        }
    )
]

# ==========================================
# 🧩 ЛОГІКА ВИКОНАННЯ ФУНКЦІЙ
# ==========================================

def fetch_ms_learn_courses(role, subject, level="beginner", min_rating=4.0, limit_minutes=300):
    """Шукає контейнерні курси з фільтрацією за часом та рейтингом."""
    print(f"[LOG] Пошук: {subject} для {role} (рівень {level})")
    
    api_url = "https://learn.microsoft.com/api/catalog/"
    query_params = {"role": role, "subject": subject, "level": level}
    
    resp = requests.get(api_url, params=query_params)
    data = resp.json()
    
    # Об'єднуємо модулі та шляхи навчання
    all_items = data.get("modules", []) + data.get("learningPaths", [])
    
    filtered_list = []
    for item in all_items:
        # Безпечне отримання рейтингу
        r_data = item.get("rating", {})
        curr_rating = r_data.get("average") if isinstance(r_data, dict) else 0
        curr_duration = item.get("duration_in_minutes", 0)
        
        # Фільтрація за нашими критеріями
        if (curr_rating and curr_rating >= min_rating) and (curr_duration <= limit_minutes):
            filtered_list.append({
                "title": item.get("title"),
                "uid": item.get("uid"),
                "rating": curr_rating,
                "duration": curr_duration
            })
    
    # Сортування за рейтингом та повернення топ-3
    filtered_list.sort(key=lambda x: x["rating"], reverse=True)
    return json.dumps(filtered_list[:3], ensure_ascii=False)

def get_full_course_details(uid):
    """Отримує детальний опис та URL курсу."""
    print(f"[LOG] Деталізація курсу: {uid}")
    api_url = f"https://learn.microsoft.com/api/catalog/?uid={uid}"
    
    resp = requests.get(api_url)
    data = resp.json()
    
    items = data.get("modules", []) or data.get("learningPaths", [])
    if not items:
        return json.dumps({"error": "Курс не знайдено"})
    
    course = items[0]
    return json.dumps({
        "name": course.get("title"),
        "desc": course.get("summary"),
        "link": "https://learn.microsoft.com" + course.get("url", ""),
        "stars": course.get("rating", {}).get("average", "N/A")
    }, ensure_ascii=False)

# Словник для мапінгу функцій
tool_registry = {
    "fetch_ms_learn_courses": fetch_ms_learn_courses,
    "get_full_course_details": get_full_course_details
}

# ==========================================
# 🔄 ОСНОВНИЙ ЦИКЛ ОБРОБКИ (HANDLER)
# ==========================================

chat_history = [
    {"role": "system", "content": "Ти — технічний експерт. Відповідай українською мовою."},
    {"role": "user", "content": (
        "Я початківець (beginner developer). Знайди для мене найкращі курси про "
        "containers (Docker/Kubernetes) на Microsoft Learn. "
        "Мене цікавлять варіанти з рейтингом 4.5+ та тривалістю до 4 годин (240 хв). "
        "Обери найкращий за рейтингом та надай його повний опис і посилання."
    )}
]

try:
    # Крок 1: Перший запит до моделі
    current_response = ai_client.complete(
        model=MODEL_ID,
        messages=chat_history,
        tools=tools_definition,
        tool_choice="auto"
    )

    # Крок 2: Цикл обробки викликів інструментів (якщо модель їх просить)
    while current_response.choices[0].message.tool_calls:
        assistant_msg = current_response.choices[0].message
        chat_history.append(assistant_msg)
        
        for call in assistant_msg.tool_calls:
            fname = call.function.name
            fargs = json.loads(call.function.arguments)
            
            # Викликаємо потрібний інструмент
            if fname in tool_registry:
                action = tool_registry[fname]
                result_data = action(**fargs)
                
                chat_history.append({
                    "role": "tool",
                    "tool_call_id": call.id,
                    "name": fname,
                    "content": result_data
                })
        
        # Отримуємо наступну відповідь з урахуванням даних від функцій
        current_response = ai_client.complete(model=MODEL_ID, messages=chat_history)

    print("\n📝 ФІНАЛЬНА ВІДПОВІДЬ АСИСТЕНТА:")
    print(current_response.choices[0].message.content)

except Exception:
    print("❌ Сталася критична помилка:")
    traceback.print_exc()

[LOG] Пошук: containers для beginner (рівень beginner)

📝 ФІНАЛЬНА ВІДПОВІДЬ АСИСТЕНТА:
На жаль, я не знайшов курсів на Microsoft Learn, що відповідають вашим критеріям (рейтинг 4.5+ і тривалість до 4 годин). Можете розглянути можливість збільшення часового ліміту або пошукати інші ресурси для навчання Docker/Kubernetes. Якщо потрібно, можу допомогти знайти курси на інших платформах.
